In [639]:
import pandas as pd
import dateutil.parser
from collections import defaultdict
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import itertools

In [640]:
e = gp.Env()
e.setParam('TimeLimit', 5*60)

model = gp.Model("AGAP", env=e)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-15
Set parameter TimeLimit to value 300


objective = minimize $w_{ijab} \cdot c_ij$

where $w_{ijab} = 1 \iff y_{ia} \vee y_{jb} \vee x_{ijab}$

to do this, first we must get the sets of valid movements.

In [641]:
flights = pd.read_csv('UA_8_1_2023.csv')
for flight in flights.itertuples():
    if dateutil.parser.parse(flight[2]) < dateutil.parser.parse(flight[1]):
        # drop this row from the df
        flights.drop(flight.Index, inplace=True)
flights = flights.iloc[:24]
gates = pd.read_csv('stands.csv')

# adding dummy gate for tarmac
gates = list(gates['0']) + ['X']

codes_unclean = list(pd.read_csv('us_airports.csv')['Code'])

us_codes = codes = [code for code in codes_unclean if len(code) == 3]
us_codes = set(us_codes)

In [642]:
gates_prime = gates

tuples = []

for a in gates_prime:
    if a == "X":
        continue
    for b in gates_prime:
        for c in gates_prime:
            if c == "X":
                continue
            tuples.append([a, b, c])


# uncomment for single gate no tows

gates_prime = gates[:-1]  # + ['C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']
np.random.shuffle(gates_prime)
tuples = [[a, a, a] for a in gates_prime]

intl_gates = []
for g in gates_prime:
    if g[0] == 'C' and int(g[1:]) <= 10:
        intl_gates.append(g)
intl_gates

['C5', 'C3', 'C7', 'C6', 'C9', 'C2', 'C4', 'C8', 'C1']

In [643]:
valid_combos = []
for tup in tuples:
    # 3 diff gates
    if len(set(tup)) == 3 and "X" not in set(tup):
        continue
    # of type [c1, c2, c1]
    if tup[0] == tup[2] and tup[1] != "X" and tup[0] != tup[1]:
        continue

    # of type [c1, c2, c2]
    if tup[0] != tup[2] and tup[1] != "X":
        continue
    else:
        valid_combos.append(tup)

Now that we have our valid movements, we can find out which valid movements are allowed to be assigned to this turn

In [644]:
flight_valid_gates = defaultdict(list)
z_ijm = []
flights_to_zijms = defaultdict(list)
international_flights = {}

for i in flights.itertuples():
    flight_movements = []

    if i[4] not in us_codes:
        international_flights[i[-2]] = True

    # Intl Origin
    if i[4] not in us_codes and (i[4][0] != 'Y' or i[4] == 'DUB' or i[4] == 'SNN' or i[4] == 'SJU'):
        # needs to be assigned an intl Gate
        for mov in valid_combos:
            if mov[0] in intl_gates:
                flight_movements.append(mov)
                flight_valid_gates[i[-1]].append(mov)
                flight_valid_gates[i[-2]].append(mov)

    # Widebody Aircraft
    elif i[3][0] == '7':
        for mov in valid_combos:
            if mov[0] in intl_gates or mov[0] in ['D1', 'D3', 'D30', "D11", 'D15', 'D19']:
                if mov[1] in intl_gates or mov[1] in ['D1', 'D3', 'D30', "D11", 'D15', 'D19'] or mov[1] == 'X':
                    if mov[2] in intl_gates or mov[2] in ['D1', 'D3', 'D30', "D11", 'D15', 'D19']:
                        flight_movements.append(mov)
                        flight_valid_gates[i[-1]].append(mov)
                        flight_valid_gates[i[-2]].append(mov)

    else:
        for mov in valid_combos:
            flight_movements.append(mov)
            flight_valid_gates[i[-1]].append(mov)
            flight_valid_gates[i[-2]].append(mov)

    f1, f2 = i[-2], i[-1]
    for mov in flight_movements:

        z_ijm.append(model.addVar(vtype=GRB.BINARY,
                     name=f"z_{f1}_{f2}_{mov[0]}_{mov[1]}_{mov[2]}"))
        flights_to_zijms[i[-1], i[-2]].append(z_ijm[-1])

model.update()

Now we can add the constraint that each flight pair operated by the same aicraft must be assigned exactly one movement.

In [645]:
for k, v in flights_to_zijms.items():
    model.addConstr(gp.quicksum(v) == 1)

If that flight is assigned that movement, then it must not conflict with any other flight's movements. so we must generate a list of conflicting movements.

In [646]:
movements = valid_combos
k2 = set()
for i in movements:
    for j in movements:
        k2.add((tuple(i), tuple(j)))

In [647]:
seen = []
overlap = []
for i in flights.itertuples():
    for j in flights.itertuples():
        if i != j and (i, j) not in seen:
            # (dep_j - arr_i)
            c1 = (dateutil.parser.parse(
                j[2]) - dateutil.parser.parse(i[1])).seconds / 60
            # (dep_i - arr_j)
            c2 = (dateutil.parser.parse(
                i[2]) - dateutil.parser.parse(j[1])).seconds / 60

            if max(dateutil.parser.parse(i[1]), dateutil.parser.parse(j[1])) < min(dateutil.parser.parse(i[2]), dateutil.parser.parse(j[2])):

                overlap.append((i, j))

            seen.append((j, i))

Now we have a list of overlapping flights. but we must see how they overlap to see which movements actually conflict.

In [648]:
deplaning = 30
boarding = 30

disallowed = []

for pair in overlap:

    # calculate time in overlap
    arrival_i, arrival_j = dateutil.parser.parse(
        pair[0][1]), dateutil.parser.parse(pair[1][1])
    departure_i, departure_j = dateutil.parser.parse(
        pair[0][2]), dateutil.parser.parse(pair[1][2])
    # arrival_i plus 30 mins
    i_done_deplaning, j_done_deplaning = arrival_i + \
        pd.Timedelta(minutes=deplaning), arrival_j + \
        pd.Timedelta(minutes=deplaning)

    i_boarding_start, j_boarding_start = departure_i - \
        pd.Timedelta(minutes=boarding), departure_j - \
        pd.Timedelta(minutes=boarding)

    deplaning_overlap, idle_overlap, boarding_overlap, ideparturejarrival, jdepartureiarrival = False, False, False, False, False

    if max(arrival_i, arrival_j) < min(i_done_deplaning, j_done_deplaning):
        deplaning_overlap = True
    if max(i_done_deplaning, j_done_deplaning) < min(i_boarding_start, j_boarding_start):
        idle_overlap = True
    if max(i_boarding_start, j_boarding_start) < min(departure_i, departure_j):
        boarding_overlap = True
    # deplaning and boarding overlap
    if max(i_boarding_start, arrival_j) < min(departure_i, j_done_deplaning):
        ideparturejarrival = True
    if max(j_boarding_start, arrival_i) < min(departure_j, i_done_deplaning):
        jdepartureiarrival = True

    this_disallowed = set()
    for tuples in k2:
        if ideparturejarrival:
            if tuples[0][2] == tuples[1][0]:
                this_disallowed.add(tuples)

        if deplaning_overlap:
            if tuples[0][0] == tuples[1][0]:
                this_disallowed.add(tuples)

        if idle_overlap:
            if tuples[0][1] == tuples[1][1] and tuples[0][1] != "X":
                this_disallowed.add(tuples)

        if boarding_overlap:
            if tuples[0][2] == tuples[1][2]:
                this_disallowed.add(tuples)

    # add constraint 4.3
    for disallowed_pair in this_disallowed:

        k = model.getVarByName(
            f"z_{pair[0][-2]}_{pair[0][-1]}_{disallowed_pair[0][0]}_{disallowed_pair[0][1]}_{disallowed_pair[0][2]}")
        kprime = model.getVarByName(
            f"z_{pair[1][-2]}_{pair[1][-1]}_{disallowed_pair[1][0]}_{disallowed_pair[1][1]}_{disallowed_pair[1][2]}")

        if not k:
            k = model.addVar(
                vtype=GRB.BINARY, name=f"z_{pair[0][-2]}_{pair[0][-1]}_{disallowed_pair[0][0]}_{disallowed_pair[0][1]}_{disallowed_pair[0][2]}")
            model.update()
            z_ijm.append(k)

        if not kprime:
            kprime = model.addVar(
                vtype=GRB.BINARY, name=f"z_{pair[1][-2]}_{pair[1][-1]}_{disallowed_pair[1][0]}_{disallowed_pair[1][1]}_{disallowed_pair[1][2]}")
            model.update()
            z_ijm.append(kprime)

        model.addConstr(k + kprime <= 1, name=k.varName + kprime.varName)

        k = model.getVarByName(
            f"z_{pair[1][-2]}_{pair[1][-1]}_{disallowed_pair[0][0]}_{disallowed_pair[0][1]}_{disallowed_pair[0][2]}")
        kprime = model.getVarByName(
            f"z_{pair[0][-2]}_{pair[0][-1]}_{disallowed_pair[1][0]}_{disallowed_pair[1][1]}_{disallowed_pair[1][2]}")

        if not k:
            k = model.addVar(
                vtype=GRB.BINARY, name=f"z_{pair[0][-2]}_{pair[0][-1]}_{disallowed_pair[0][0]}_{disallowed_pair[0][1]}_{disallowed_pair[0][2]}")
            model.update()
            z_ijm.append(k)

        if not kprime:
            kprime = model.addVar(
                vtype=GRB.BINARY, name=f"z_{pair[1][-2]}_{pair[1][-1]}_{disallowed_pair[1][0]}_{disallowed_pair[1][1]}_{disallowed_pair[1][2]}")
            model.update()
            z_ijm.append(kprime)

        model.addConstr(k + kprime <= 1, name=k.varName + kprime.varName)

At the same time, if a flight is assigned that movement, then the first flight must be assigned to the first gate in that movement and the second must be assigned to the second gate.

In [649]:
flt_to_y = defaultdict(list)
yvars = []


for flight in flights.itertuples():

    for g in gates_prime:
        flt_to_y[flight[-2]
                 ].append(model.addVar(vtype=GRB.BINARY, name=f"y_{flight[-2]}_{g}"))
        flt_to_y[flight[-1]
                 ].append(model.addVar(vtype=GRB.BINARY, name=f"y_{flight[-1]}_{g}"))


model.update()

# add constraint 4.7
for gateset in flt_to_y.values():
    model.addConstr(gp.quicksum(gateset) == 1)

In [650]:
for z in z_ijm:

    seq = z.varName.split('_')
    # make this if and only if
    if not model.getVarByName(f"y_{seq[1]}_{seq[3]}"):
        print(f"y_{seq[1]}_{seq[3]}")

    # add constraints 4.4-6
    model.addConstr(z <= model.getVarByName(
        f"y_{seq[1]}_{seq[3]}"),  name=f" z leq y_{seq[1]}_{seq[3]}")
    model.addConstr(z <= model.getVarByName(
        f"y_{seq[2]}_{seq[5]}"),  name=f" z leq y_{seq[2]}_{seq[5]}")
    model.addConstr(z >= model.getVarByName(f"y_{seq[1]}_{seq[3]}") + model.getVarByName(
        f"y_{seq[2]}_{seq[5]}") - 1,  name=f" z geq y_{seq[1]}_{seq[3]}")

Now we have to set $x_{ijab}$ for all valid pairs of connections.

In [651]:
connections = []

# generate list of feasible connections
for i in flights.iterrows():
    for j in flights.iterrows():
        if i[1]["Arrival Time"] < j[1]["Departure Time"]:
            connections.append([i[1]["code"], j[1]["outgoing code"]])
        if j[1]["Arrival Time"] < i[1]["Departure Time"]:
            connections.append([j[1]["code"], i[1]["outgoing code"]])

# generate number of pax on each connections
cnx_num_pax = {tuple(connection): np.random.randint(1, 12)
               for connection in connections}

connections = list(cnx_num_pax.keys())

In [652]:
xijabs = []
for connection in connections:
    for g1 in gates_prime[:-1]:
        for g2 in gates_prime[:-1]:

            xijabs.append(model.addVar(vtype=GRB.INTEGER,
                          name=f"x_{connection[0]}_{connection[1]}_{g1}_{g2}"))

model.update()

Now we need to get our connection matrix - time available in between flights

In [653]:
S = {}

deplaning_time = 15
cutoff_time = 15


for i in connections:

    obd_dep = str(
        list(flights[flights['outgoing code'] == i[1]]['Departure Time'])[0])
    inc_arr = str(list(flights[flights['code'] == i[0]]['Arrival Time'])[0])

    S[i[0], i[1]] = (dateutil.parser.parse(
        obd_dep) - dateutil.parser.parse(inc_arr)).seconds / 60 - (deplaning_time + cutoff_time)

And the same for time to transit between gates 

In [654]:
international_flights

{73: True, 1562: True, 997: True, 860: True}

In [655]:
wait_times = pd.read_excel("CBP_Wait_Times.xlsx")[['Hour', 'All']]

# drop first 3 rows
wait_times = wait_times.iloc[3:]
wait_times

# split the hour column into two columns, delimeter is the dash inside the cells
wait_times[['start', 'end']] = wait_times['Hour'].str.split("-", expand=True)
wait_times = wait_times.drop(columns=['Hour'])

wait_times['start'].astype(int)

wait_times = wait_times[['start', 'All']]
wait_times = wait_times.set_index('start')
wait_times_dict = wait_times.to_dict()["All"]

wait_times_dict['1500 ']

48

The below code takes the longest to run, and would benefit greatly from multiprocessing.

In [656]:
t_ab = {}

for i in connections:
    for a in gates_prime:
        for b in gates_prime:
            pre_a, suf_a = a[0], int(a[1:])
            pre_b, suf_b = b[0], int(b[1:])

            if pre_a == 'D':
                suf_a = suf_a + 28
            if pre_b == 'D':
                suf_b = suf_b + 28

            intl_adj = 0

            if i[0] in international_flights:

                # query flights to get arrival time of flight i[0]
                arr_time = dateutil.parser.parse(
                    str(list(flights[flights['code'] == i[0]]['Arrival Time'])[0]))
                if arr_time.hour * 100 < 1000:

                    intl_adj = wait_times_dict["0" +
                                               str(arr_time.hour * 100)+" "]
                else:
                    try:
                        intl_adj = wait_times_dict[str(
                            arr_time.hour * 100)+" "]
                    except KeyError:
                        intl_adj = 60

            t_ab[str(i[0]), a, b] = intl_adj + abs(suf_a - suf_b)

now we need to add

In [657]:
for x in xijabs:
    seq = x.varName.split('_')

    model.addConstr(S[float(seq[1]), float(seq[2])] >=
                    t_ab[(seq[1], seq[3], seq[4])] - (10000000 * x))

If and only if x_ijab and y_ia and y_jb are all 1, then w_ijab = 1.

In [658]:
x_to_w = {}
for x in xijabs:
    x_to_w[x] = model.addVar(
        vtype=GRB.BINARY, name=f"w_{'_'.join(x.varName.split('_')[1:])}")

model.update()

for x in xijabs:
    seq = x.varName.split('_')
    y1 = model.getVarByName(f"y_{seq[1]}_{seq[3]}")
    y2 = model.getVarByName(f"y_{seq[2]}_{seq[4]}")

    model.addConstr(y1 + y2 + x - 2 <= x_to_w[x])

In [659]:
ws = list(x_to_w.values())

In [660]:
w_to_cnx = {}

for w in ws:

    w_to_cnx[w] = tuple(int(i) for i in w.VarName.split('_')[1:3])

In [661]:
# model.setObjective(0, GRB.MAXIMIZE)

In [662]:
model.setObjective(gp.quicksum(
    cnx_num_pax[w_to_cnx[w]] * w for w in ws), GRB.MINIMIZE)

In [663]:
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2121464 rows, 2097992 columns and 5294186 nonzeros
Model fingerprint: 0x189c2de7
Variable types: 0 continuous, 2097992 integer (1050616 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 2017114 rows and 2017114 columns
Presolve time: 0.58s
Presolved: 104350 rows, 80878 columns, 290220 nonzeros
Variable types: 0 continuous, 80878 integer (80878 binary)
Found heuristic solution: objective 18.0000000
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 0.000000e+00, 484 iterations, 0.12 seconds (0.16 work units)

    Nodes    |    Curren

In [664]:
gateassignments = []
for v in model.getVars():
    if ('z' in v.VarName) and v.x == 1:

        seq = v.varName.split('_')
        if 'z' in v.varName:
            gateassignments.append([seq[1], seq[2], seq[3]])

In [665]:
assignment = pd.DataFrame(gateassignments, columns=[
                          "Incoming Flight", "Outgoing Flight", "Gate"])
# assignment.to_csv('gateassignments_Optimized_51_75.csv', index=False)
assignment

,Incoming Flight,Outgoing Flight,Gate
0,783,2021,D12
1,2626,1154,D4
2,584,1964,D2
3,2050,380,C3
4,1395,1600,D6
5,73,700,C9
6,2046,277,C1
7,1562,618,C5
8,997,1744,C2
9,860,2058,C7


In [666]:
cnxpax = 0
for x in xijabs:
    if x.x > 0:
        seq = x.varName.split('_')
        if model.getVarByName(f"y_{seq[1]}_{seq[3]}").x == 1 and model.getVarByName(f"y_{seq[2]}_{seq[4]}").x == 1:
            cnxpax += cnx_num_pax[int(x.varName.split('_')[1]),
                                  int(x.varName.split('_')[2])]

In [667]:
cnxpax

0